In [328]:
import datetime as dt
import numpy as np
import pandas as pd
import talib
import yfinance as yf
import vectorbt as vbt
import matplotlib.pyplot as plt
import pandas as pd
import time
import tqdm
import os

In [329]:
def load_tickers_df_from_jpx_xls(path, code_col):
    df = pd.read_excel(path)
    codes = (
        df[code_col]
        .astype(str)
        .str.replace(r"\D", "", regex=True)    # 数字以外除去
        .str.zfill(4)                          # 4桁ゼロ埋め
    )
    df["code_ticker"] = [i+".T" for i in codes]
    return df

def bach_download(tickers, start, end, batch_size=50, sleep_sec=1.0):
    all_open = []
    all_close = []
    good = []

    for i in tqdm.tqdm(range(0, len(tickers), batch_size)):
        batch = tickers[i:i+batch_size]
        try:
            df = yf.download(batch, start=start, end=end, group_by='ticker', auto_adjust=True, threads=True)
            if isinstance(df.columns, pd.MultiIndex):
                # MultiIndex: 各銘柄ごとに Open/Close を抜き出す
                batch_close = pd.DataFrame({t: df[t]['Close'] for t in batch if t in df.columns.get_level_values(0)})
                batch_open  = pd.DataFrame({t: df[t]['Open']  for t in batch if t in df.columns.get_level_values(0)})
            else:
                # 単一銘柄（list長=1）ケース
                t = batch[0]
                batch_close = pd.DataFrame({t: df['Close']})
                batch_open  = pd.DataFrame({t: df['Open']})

            # 全欠損・極端にデータが短い銘柄は落とす
            valid_cols = [c for c in batch_close.columns if batch_close[c].dropna().shape[0] >= max(L_PERIOD+5, 40)]
            batch_close = batch_close[valid_cols]
            batch_open  = batch_open[valid_cols]

            if len(valid_cols) > 0:
                all_close.append(batch_close)
                all_open.append(batch_open)
                good.extend(valid_cols)
        except Exception as e:
            # バッチ単位で失敗しても継続
            print(f"[WARN] batch {i}-{i+batch_size} failed: {e}")

        time.sleep(sleep_sec)

    if not all_close:
        raise RuntimeError("価格データが取得できませんでした。銘柄リストや期間を見直してください。")

    # 列方向で結合（銘柄を増やす）
    close_df = pd.concat(all_close, axis=1).sort_index()
    open_df  = pd.concat(all_open, axis=1).sort_index()

    # 列の重複を削除
    close_df = close_df.loc[:, ~close_df.columns.duplicated()]
    open_df  = open_df.loc[:, ~open_df.columns.duplicated()]

    return close_df, open_df, sorted(set(good))

def calc_cross(close_df, s_period, l_period):
    sma_s = close_df.rolling(s_period).mean()
    sma_l = close_df.rolling(l_period).mean()

    gc = sma_s > sma_l
    dc = sma_s < sma_l

    entries = (gc & ~gc.shift(1, fill_value=False)).shift(1, fill_value=False)
    exits   = (dc & ~dc.shift(1, fill_value=False)).shift(1, fill_value=False)

    # NaN期間での誤シグナル抑止
    entries = entries & sma_s.notna() & sma_l.notna()
    exits   = exits   & sma_s.notna() & sma_l.notna()
    return sma_s, sma_l, entries, exits

# 設定値

In [330]:
S_PERIOD = 5
L_PERIOD = 25
# 片道コストとスリッページ（例：0.10% = 0.001）
fees = 0.001
slip = 0.0
INIT_CASH = 1_000_000
TOP_N = 10

start = "2023-08-01"
end = dt.date.today().isoformat()

# 実行(株コード取得>データ取得>バックテスト>結果)

## 株コード取得

In [331]:
ticker_df = load_tickers_df_from_jpx_xls("data_j.xls", "コード")
ticker_df = ticker_df[ticker_df["33業種区分"]=="食料品"] #コメントアウトしない場合サンプルとして4000ある株を食料品関係に絞り込み
ticker_df_code = list(ticker_df["code_ticker"].values)

print("該当株数", len(ticker_df_code))

該当株数 127


## データ取得  

In [332]:
#ticker_df_food_code = ticker_df_food_code[:10] #コメントアウトしない場合サンプルとしてさらに取得順で10個に絞り込み
close_df, open_df, good = bach_download(ticker_df_code, start, end) #50個ずつ取得

print("正常に取得できたデータ数", len(close_df.columns.get_level_values(0)))

[*********************100%***********************]  50 of 50 completed                           | 0/3 [00:00<?, ?it/s]

2 Failed downloads:
['0243.T', '0226.T']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  50 of 50 completed                   | 1/3 [00:03<00:06,  3.01s/it]

1 Failed download:
['0250.T']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  27 of 27 completed                   | 2/3 [00:05<00:02,  2.65s/it]

1 Failed download:
['0409.T']: YFTzMissingError('possibly delisted; no timezone found')
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:07<00:00,  2.55s/it]

正常に取得できたデータ数 123


## バックテスト

In [333]:
# 移動平均計算、クロス計算
sma_s, sma_l, entries, exits = calc_cross(close_df, S_PERIOD, L_PERIOD)
# バックテスト実施
pf = vbt.Portfolio.from_signals(
    close=close_df,
    entries=entries,
    exits=exits,
    open=open_df,             # 寄りで約定
    init_cash=INIT_CASH,
    fees=fees,
    slippage=slip,
    freq="1D",
)
# 利率でソートしてトップNを計算
ret = pf.total_return().sort_values(ascending=False)
topN = ret.head(TOP_N)

# 結果

In [334]:
os.makedirs("出力", exist_ok=True)
os.makedirs("出力/グラフ", exist_ok=True)

# Total Returnを票で出力
## total_return
ret_df = pd.DataFrame(ret)
ret_df["key"] = ret_df.index
ret_df_merge = pd.merge(ret_df, ticker_df, left_on="key", right_on="code_ticker", how="left")
ret_df_merge = ret_df_merge.drop(columns="key")
ret_df_merge.to_csv("出力/利益率.csv", encoding="utf_8_sig")

# topNの詳細情報を表で出力
## バックテスト結果
stats = {}
for i in tqdm.tqdm(topN.index):
    stats[i] = pf[i].stats()
topNData = pd.DataFrame(stats).T
topNData["Total Return"] = topNData["Total Return [%]"]*INIT_CASH
topNData["key"] = topNData.index
## 株情報と結合
stats_df = pd.merge(topNData, ticker_df, left_on="key", right_on="code_ticker", how="left")
output_stats_df = stats_df.drop(columns="key")
output_stats_df.to_csv("出力/TOPN.csv", encoding="utf_8_sig")

# topNのグラフを出力
for i, x in enumerate(tqdm.tqdm(output_stats_df.index)):
    pf[x].plot().write_html("出力/グラフ/{}_{}.html".format(x, output_stats_df.loc[i,"銘柄名"]))

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]C:\Users\akari\Desktop\kabu\.venv\Lib\site-packages\vectorbt\base\indexing.py:145: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

 10%|████████▎                                                                          | 1/10 [00:00<00:01,  4.63it/s]C:\Users\akari\Desktop\kabu\.venv\Lib\site-packages\vectorbt\base\indexing.py:145: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

 20%|████████████████▌                                                                  | 2/10 [00:00<00:01,  4.30it/s]C:\Users\

In [335]:
# ここでも出力
output_stats_df

,Start,End,Period,Start Value,End Value,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],...,コード,銘柄名,市場・商品区分,33業種コード,33業種区分,17業種コード,17業種区分,規模コード,規模区分,code_ticker
0,2023-08-01 00:00:00,2025-10-17 00:00:00,542 days 00:00:00,1000000.0,2905953.181172,190.595318,303.346708,100.0,75813.240911,45.821454,...,2901,ウェルディッシュ,スタンダード（内国株式）,3050,食料品,1,食品,-,-,2901.T
1,2023-08-01 00:00:00,2025-10-17 00:00:00,542 days 00:00:00,1000000.0,2140770.333,114.077033,208.461802,100.0,60951.766069,33.874929,...,2820,やまみ,スタンダード（内国株式）,3050,食料品,1,食品,-,-,2820.T
2,2023-08-01 00:00:00,2025-10-17 00:00:00,542 days 00:00:00,1000000.0,1823655.443827,82.365544,150.44511,100.0,39994.564392,21.429993,...,2585,ライフドリンク カンパニー,プライム（内国株式）,3050,食料品,1,食品,7,TOPIX Small 2,2585.T
3,2023-08-01 00:00:00,2025-10-17 00:00:00,542 days 00:00:00,1000000.0,1779331.811896,77.933181,140.972206,100.0,35166.062684,29.789414,...,2938,オカムラ食品工業,スタンダード（内国株式）,3050,食料品,1,食品,-,-,2938.T
4,2023-08-01 00:00:00,2025-10-17 00:00:00,542 days 00:00:00,1000000.0,1704097.115777,70.409712,110.868662,100.0,34747.510658,16.474881,...,2903,シノブフーズ,スタンダード（内国株式）,3050,食料品,1,食品,-,-,2903.T
5,2023-08-01 00:00:00,2025-10-17 00:00:00,542 days 00:00:00,1000000.0,1652608.536496,65.260854,127.918469,100.0,25170.679282,27.337852,...,2112,塩水港精糖,スタンダード（内国株式）,3050,食料品,1,食品,-,-,2112.T
6,2023-08-01 00:00:00,2025-10-17 00:00:00,542 days 00:00:00,1000000.0,1587491.253661,58.749125,105.07261,100.0,33223.403106,21.868855,...,2501,サッポロホールディングス,プライム（内国株式）,3050,食料品,1,食品,4,TOPIX Mid400,2501.T
7,2023-08-01 00:00:00,2025-10-17 00:00:00,542 days 00:00:00,1000000.0,1545310.196677,54.53102,77.752524,100.0,33465.471112,12.488835,...,2914,日本たばこ産業,プライム（内国株式）,3050,食料品,1,食品,1,TOPIX Core30,2914.T
8,2023-08-01 00:00:00,2025-10-17 00:00:00,542 days 00:00:00,1000000.0,1531733.730378,53.173373,131.344725,100.0,31390.417756,14.552642,...,2540,養命酒製造,プライム（内国株式）,3050,食料品,1,食品,7,TOPIX Small 2,2540.T
9,2023-08-01 00:00:00,2025-10-17 00:00:00,542 days 00:00:00,1000000.0,1516771.282887,51.677128,88.570814,100.0,34414.417157,15.211359,...,2809,キユーピー,プライム（内国株式）,3050,食料品,1,食品,4,TOPIX Mid400,2809.T
